Parte 1: Validación de Datos Q3 2025

In [0]:
%sql
-- Parte A: Conteo de registros por tabla
SELECT 
(SELECT COUNT(*) FROM workspace.operations.gold_fact_sales_product) AS total_transacciones,
(SELECT COUNT(*) FROM workspace.operations.gold_dim_product WHERE STATUS = 'A') AS total_productos_activos,
(SELECT COUNT(*) FROM workspace.operations.gold_dim_organization WHERE STORE_ACTIVE = 'Y') AS total_tiendas_activas,
(SELECT COUNT(*) FROM workspace.operations.gold_dim_supplier WHERE SUP_STATUS = 'A') AS total_proveedores_activos

Hay 432987 transacciones, 749 productos activos, 94 tiendas activas y 7 proveedores activos.

In [0]:
%sql
--Parte B: Validación de rangos por fecha
SELECT MIN(BUSINESS_DATE) AS fecha_minima, MAX(BUSINESS_DATE) AS fecha_maxima, COUNT (DISTINCT BUSINESS_DATE) AS total_dias FROM workspace.operations.gold_fact_sales_product

In [0]:
%sql
--Parte C: Validación de integridad referencial
SELECT 
(SELECT COUNT(*) FROM workspace.operations.gold_fact_sales_product f LEFT JOIN workspace.operations.gold_dim_product d ON f.SK_PRODUCT = d.SK_PRODUCT WHERE d.SK_PRODUCT IS NULL) AS total_productos_huerfanos,
(SELECT COUNT(*) FROM workspace.operations.gold_fact_sales_product f LEFT JOIN workspace.operations.gold_dim_organization o ON f.SK_ORGANIZATION = o.SK_ORGANIZATION WHERE o.SK_ORGANIZATION IS NULL) AS total_tiendas_huerfanas,
(SELECT COUNT(*) FROM workspace.operations.gold_fact_sales_product f LEFT JOIN workspace.operations.gold_dim_supplier s ON f.SK_SUPPLIER = s.SK_SUPPLIER WHERE s.SK_SUPPLIER IS NULL) AS total_proveedores_huerfanos,
(SELECT COUNT(*) FROM workspace.operations.gold_fact_sales_product f LEFT JOIN workspace.operations.gold_dim_channel c ON f.SK_CHANNEL = c.SK_CHANNEL WHERE c.SK_CHANNEL IS NULL) AS total_canal_huerfanos,
(SELECT COUNT(*) FROM workspace.operations.gold_fact_sales_product f LEFT JOIN workspace.operations.gold_dim_currency cu ON f.SK_CURRENCY = cu.SK_CURRENCY WHERE cu.SK_CURRENCY IS NULL) AS total_moneda_huerfanos

In [0]:
%sql
--Parte D: Validación de valores nulos y negativos 
SELECT COUNT(*) AS total_registros, 
SUM(CASE WHEN TOTAL_SALE IS NULL THEN 1 ELSE 0 END) as ventas_nulas,
SUM (CASE WHEN TOTAL_SALE < 0 THEN 1 ELSE 0 END) as ventas_negativas,
SUM(CASE WHEN UNITS <= 0 OR UNITS IS NULL THEN 1 ELSE 0 END) as unidades_invalidas
FROM workspace.operations.gold_fact_sales_product

In [0]:
%sql
-- Parte E: Validación de consistencia de cálculos 
SELECT COUNT(*) AS filas_error
FROM workspace.operations.gold_fact_sales_product
WHERE ABS((TOTAL_SALE + TAX )- TOTAL_SALE_TAX) > 0.01 --Por redondeos decimales

Parte 1.2 Documento de Hallazgos
-Validaciones:
1. Rango de fechas: Se confirmó que la data cubre del 2025-07-01 al 2025-09-30 (92 días). 
2. Integridad Referencial: No se encontraron registros huérfanos. Todas las ventas tienen dimensiones asociadas. 
3. Validación de valores nulos y negativos: No se encontraron ventas nulas o negativas, ni unidades inválidas. 
4. Validación de consistencia de calculos: El cálculo del impuesto coincide con el total de ventas en todos los registros. Se aplicó un margen de tolerancia de 0.01 para omitir variaciones de números decimales. 

- Problemas encontrados: 
1. No se encontraron problemas en el dataset proporcionado para el periodo Q3 2025. 

- Recomendaciones: 
1. En este caso toda la data estuvo correcta y sin anomalías, para seguir manteniendo esta calidad, hice un poco de investigación previa a la prueba y vi que existe un feature en Databricks llamado Databricks SQL Alerts que monitorea la data periódicamente y puede alertar cuando se ingresan valores negativos/nulos.